In [ ]:
#pip install pandas faker

In [ ]:
import pandas as pd
import random
from faker import Faker
fake = Faker()


In [ ]:
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Number of records to generate
num_records = 200

# Placeholder for subtype-specific EmployeeIDs
employee_data = {
    "AirplaneStaff": [],
    "SecurityStaff": [],
    "GroundStaff": []
}

# 1. Airport Table
def generate_airport_sql():
    return [
        f"INSERT INTO Airport (AirportName, [Type], City, Area, Country) VALUES ('{fake.city()} Airport', "
        f"'{random.choice(['International', 'Domestic'])}', '{fake.city()}', '{fake.street_name()}', '{fake.country()}');"
        for _ in range(num_records)
    ]

# 2. Airline Table
def generate_airline_sql():
    return [
        f"INSERT INTO Airline (AirlineName, Country) VALUES ('{fake.company()}', '{fake.country()}');"
        for _ in range(num_records)
    ]

# 3. Passenger Table
def generate_passenger_sql():
    return [
        f"INSERT INTO Passenger (FirstName, LastName, Age, Nationality, Email, CustomerFeedback) VALUES "
        f"('{fake.first_name()}', '{fake.last_name()}', {random.randint(18, 80)}, '{fake.country()}', "
        f"'{fake.email()}', '{random.choice(['Excellent', 'Good', 'Average', 'Poor', 'Terrible'])}');"
        for _ in range(num_records)
    ]

# 4. Phone Table
def generate_phone_sql():
    def generate_phone_number(phone_type):
        """Generate a phone number of the appropriate length based on the phone type."""
        if phone_type == "Home":
            length = random.randint(8, 15)
        elif phone_type == "Work" or phone_type == "Mobile":
            length = random.randint(10, 13)
        else:
            length = 10  # Default fallback (shouldn't happen)
        # Generate a random phone number with the specified length
        return ''.join(str(random.randint(0, 9)) for _ in range(length))

    return [
        f"INSERT INTO Phone (PassengerID, PhoneType, PhoneNumber) VALUES "
        f"({random.randint(3000, 3199)}, '{phone_type}', '{generate_phone_number(phone_type)}');"
        for phone_type in random.choices(["Home", "Work", "Mobile"], k=num_records)
    ]


# 5. LoyaltyProgram Table
def generate_loyalty_program_sql():
    return [
        f"INSERT INTO LoyaltyProgram (PassengerID, LoyaltyPoints, LoyaltyStatus, EnrollmentDate) VALUES "
        f"({random.randint(3000, 3199)}, {random.randint(0, 5000)}, "
        f"'{random.choice(['Silver', 'Gold', 'Platinum'])}', "
        f"'{fake.date_time_this_decade().strftime('%Y-%m-%d %H:%M:%S')}');"
        for _ in range(num_records)
    ]

# 6. Employee Table
def generate_employee_sql():
    sql_statements = []
    employee_id_counter = 10000  # Start EmployeeID counter

    for employee_type in ["AirplaneStaff", "GroundStaff", "SecurityStaff"]:
        num_records_per_type = num_records // 3
        for _ in range(num_records_per_type):
            first_name = fake.first_name()
            last_name = fake.last_name()
            salary = round(random.uniform(50000, 150000), 2)
            
            sql_statements.append(
                f"INSERT INTO Employee (FirstName, LastName, Salary, Employee_Type) VALUES "
                f"('{first_name}', '{last_name}', {salary}, '{employee_type}');"
            )
            employee_data[employee_type].append(employee_id_counter)
            employee_id_counter += 1

    return sql_statements


# 7. AirplaneStaff Table
def generate_airplane_staff_sql():
    return [
        f"INSERT INTO AirplaneStaff (EmployeeID, AirlineID, Position, LicenseNumber, YearsOfExperience, FlightHours, MedicalClearance) VALUES "
        f"({employee_id}, {random.randint(100, 299)}, '{random.choice(['Pilot', 'Co-Pilot', 'Flight Attendant', 'Cabin Crew'])}', "
        f"'{fake.uuid4()[:8]}', {random.randint(0, 20)}, {round(random.uniform(100, 5000), 1)}, '{fake.date_this_decade()}');"
        for employee_id in employee_data["AirplaneStaff"]
    ]

# 8. Specialization Table
def generate_specialization_sql():
    return [
        f"INSERT INTO Specialization (EmployeeID, Specialization, CertificationDate) VALUES "
        f"({random.choice(employee_data['AirplaneStaff'])}, '{random.choice(['Pilot Training', 'Emergency Procedures', 'Safety Protocols', 'Customer Service'])}', "
        f"'{fake.date_this_decade()}');"
        for _ in range(num_records)
    ]

# 9. Certification Table
def generate_certification_sql():
    return [
        f"INSERT INTO Certification (EmployeeID, CertificationLevel, IssueDate, ExpiryDate) VALUES "
        f"({random.choice(employee_data['AirplaneStaff'])}, '{random.choice(['Pilot License', 'Safety Training', 'Emergency Procedures'])}', "
        f"'{fake.date_this_decade()}', '{fake.date_between(start_date='+1y', end_date='+5y')}');"
        for _ in range(num_records)
    ]

# 10. SecurityStaff Table
def generate_security_staff_sql():
    return [
        f"INSERT INTO SecurityStaff (EmployeeID, AirportID, AssignedArea, SecurityClearanceLevel, TrainingCompleted, ShiftStart, ShiftEnd, Role) VALUES "
        f"({employee_id}, {random.randint(1, 25)}, '{random.choice(['Baggage Handling', 'Terminal Security', 'Customs', 'Runway Access'])}', "
        f"'{random.choice(['Low', 'Medium', 'High', 'Top Secret'])}', "
        f"'{fake.date_this_decade().strftime('%Y-%m-%d %H:%M:%S')}', "
        f"'{(shift_start := fake.date_time_this_year()).strftime('%Y-%m-%d %H:%M:%S')}', "
        f"'{(shift_end := shift_start + timedelta(hours=random.randint(1, 8))).strftime('%Y-%m-%d %H:%M:%S')}', "
        f"'{random.choice(['Security Officer', 'Customs Officer'])}');"
        for employee_id in employee_data["SecurityStaff"]
    ]

# 11. GroundStaff Table
def generate_ground_staff_sql():
    return [
        f"INSERT INTO GroundStaff (EmployeeID, AirportID, AssignedTasks, ShiftGroup) VALUES "
        f"({employee_id}, {random.randint(1, 25)}, '{random.choice(['Equipment Operation', 'Maintenance Support', 'Loading/Unloading', 'Fueling Assistance'])}', "
        f"'{random.choice(['A', 'B', 'C', 'D'])}');"
        for employee_id in employee_data["GroundStaff"]
    ]

# 12. GroundSupport Table
def generate_ground_support_sql():
    return [
        f"INSERT INTO GroundSupport (EquipmentType, GSM_Status, UsageDuration, MaintenanceFrequency) VALUES "
        f"('{random.choice(['Baggage Cart', 'Fuel Truck', 'Tow Tractor', 'Ground Power Unit'])}', '{random.choice(['Operational', 'Out of Service'])}', "
        f"'{str(timedelta(hours=random.randint(1, 24)))}', '{random.choice(['Monthly', 'Yearly'])}');"
        for _ in range(num_records)
    ]

# 13. EquipmentAssignment Table
def generate_equipment_assignment_sql():
    return [
        f"INSERT INTO EquipmentAssignment (EquipmentID, EmployeeID, Assign_Date) VALUES "
        f"({random.randint(440, 639)}, {employee_id}, '{fake.date_time_this_year().strftime('%Y-%m-%d %H:%M:%S')}');"
        for employee_id in employee_data["GroundStaff"]
    ]


# 14. RunwayManagement Table
def generate_runway_management_sql():
    return [
        f"INSERT INTO RunwayManagement (AirportID, RunwayNumber, OccupiedStatus) VALUES "
        f"({random.randint(1, 25)}, 'Runway {random.randint(1, 50)}', '{random.choice(['Available', 'Occupied', 'Not in Use'])}');"
        for _ in range(num_records)
    ]

# 15. Aircraft Table
def generate_aircraft_sql():
    return [
        f"INSERT INTO Aircraft (AirlineID, RunwayID, AircraftType, Capacity, FuelConsumption, AircraftAge) VALUES "
        f"({random.randint(100, 299)}, {random.randint(1, 50)}, '{random.choice(['Boeing 737', 'Airbus A320', 'Bombardier CRJ'])}', "
        f"{random.randint(100, 300)}, {round(random.uniform(1000, 5000), 2)}, {random.randint(1, 30)});"
        for _ in range(num_records)
    ]

# 16. Maintenance Table
def generate_maintenance_sql():
    return [
        f"INSERT INTO Maintenance (AircraftID, EquipmentID, MaintenanceDate, MaintenanceType, MaintenanceDuration, NextScheduledMaintenance, MaintenanceStatus) VALUES "
        f"({random.randint(1, 199)}, {random.randint(440, 639)}, '{fake.date_this_year()}', '{random.choice(['Routine Check', 'Engine Check', 'Hydraulic System Maintenance'])}', "
        f"'{str(timedelta(hours=random.randint(1, 8)))}', '{fake.date_this_decade()}', '{random.choice(['Pending', 'In Progress', 'Completed'])}');"
        for _ in range(num_records)
    ]

# 17. Gate Table
def generate_gate_sql():
    return [
        f"INSERT INTO Gate (AirportID, GateNumber, Terminal, GateStatus) VALUES "
        f"({random.randint(1, 25)}, '{chr(65 + random.randint(0, 25))}{random.randint(1, 50)}', '{random.choice(['Terminal 1', 'Terminal 2', 'Terminal 3'])}', "
        f"'{random.choice(['Occupied', 'Available'])}');"
        for _ in range(num_records)
    ]

# 18. Flight Table
def generate_flight_sql(num_records):
    flight_numbers = set()  # To ensure unique flight numbers
    inserts = []

    for _ in range(num_records):
        # Generate Source and Destination Airports, ensuring they're different
        source = random.randint(1, 25)
        dest = random.choice([id for id in range(1, 26) if id != source])

        # Generate other foreign keys
        airline = random.randint(100, 299)
        runway = random.randint(1, 50)
        gate = random.randint(1, 50)  # Assuming valid GateIDs exist from 1 to 50
        aircraft = random.randint(1, 199)

        # Generate a unique flight number
        flight_number = f"FL{random.randint(1000, 9999)}"
        while flight_number in flight_numbers:
            flight_number = f"FL{random.randint(1000, 9999)}"
        flight_numbers.add(flight_number)

        # Generate realistic Departure and Arrival times
        departure_time = fake.date_time_this_year()
        arrival_time = departure_time + timedelta(hours=random.randint(1, 12))

        # Generate flight status and set dependent attributes
        flight_status = random.choice(['Scheduled', 'Delayed', 'Cancelled', 'Completed'])
        actual_departure = departure_time if flight_status == 'Completed' else None
        actual_arrival = arrival_time if flight_status == 'Completed' else None
        delay_reason = random.choice(['Weather Delay', 'Technical Issue', 'Air Traffic']) if flight_status == 'Delayed' else None

        # Append SQL INSERT statement
        inserts.append(
            f"INSERT INTO Flight (SourceAirportID, DestinationAirportID, AirlineID, RunwayID, AircraftID, GateID, "
            f"FlightNumber, FlightName, Capacity, DepartureTime, ArrivalTime, FlightPrice, FlightStatus, "
            f"ActualDepartureTime, ActualArrivalTime, DelayReason) VALUES "
            f"({source}, {dest}, {airline}, {runway}, {aircraft}, {gate}, '{flight_number}', '{fake.company()}', "
            f"{random.randint(100, 300)}, '{departure_time.strftime('%Y-%m-%d %H:%M:%S')}', "
            f"'{arrival_time.strftime('%Y-%m-%d %H:%M:%S')}', {round(random.uniform(100, 1500), 2)}, "
            f"'{flight_status}', "
            f"{'NULL' if not actual_departure else f"'{actual_departure.strftime('%Y-%m-%d %H:%M:%S')}'"}, "
            f"{'NULL' if not actual_arrival else f"'{actual_arrival.strftime('%Y-%m-%d %H:%M:%S')}'"}, "
            f"{'NULL' if not delay_reason else f"'{delay_reason}'"});"
        )
    return inserts


# 19. FlightBooking Table
def generate_flight_booking_sql():
    return [
        f"INSERT INTO FlightBooking (FlightID, PassengerID, PurchaseDate, FareClass, BookingStatus, TicketPrice) VALUES "
        f"({random.randint(1, 199)}, {random.randint(3000, 3199)}, "
        f"'{fake.date_time_this_year().strftime('%Y-%m-%d %H:%M:%S')}', "
        f"'{random.choice(['Economy', 'Business', 'First'])}', "
        f"'{random.choice(['Confirmed', 'Cancelled', 'Pending'])}', "
        f"{round(random.uniform(100, 1500), 2)});"
        for _ in range(num_records)
    ]


# 20. BoardingPass Table
def generate_boarding_pass_sql():
    return [
        f"INSERT INTO BoardingPass (PassengerID, TicketID, SeatNumber, BoardingGroup, IssueTime) VALUES "
        f"({random.randint(3000, 3199)}, {random.randint(1, 199)}, "
        f"'{random.randint(1, 50)}{chr(65 + random.randint(0, 5))}', "
        f"'{random.choice(['A', 'B', 'C', 'D'])}', "
        f"'{fake.date_time_this_year().strftime('%Y-%m-%d %H:%M:%S')}');"
        for _ in range(num_records)
    ]

# 21. Luggage Table
def generate_luggage_sql():
    return [
        f"INSERT INTO Luggage (AirportID, TicketID, PassengerID, NumberOfBags, Weight, Status, BaggageLocation) VALUES "
        f"({random.randint(1, 25)}, {random.randint(1, 199)}, {random.randint(3000, 3199)}, {random.randint(1, 5)}, {round(random.uniform(10, 50), 2)}, "
        f"'{random.choice(['Checked In', 'In Transit', 'Delivered', 'Lost'])}', '{fake.street_name()}');"
        for _ in range(num_records)
    ]

# 22. ParkingSlots Table
def generate_parking_slots_sql():
    return [
        f"INSERT INTO ParkingSlots (AirportID, AvailabilityStatus) VALUES "
        f"({random.randint(1, 25)}, '{random.choice(['Available', 'Occupied'])}');"
        for _ in range(num_records)
    ]

# 23. Parking Table
def generate_parking_sql():
    return [
        f"INSERT INTO Parking (SlotNumber, PassengerID, CheckInTime, CheckOutTime, ParkingFee) VALUES "
        f"({random.randint(100, 300)}, {random.randint(3000, 3199)}, "
        f"'{(check_in_time := fake.date_time_this_year()).strftime('%Y-%m-%d %H:%M:%S')}', "
        f"'{(check_in_time + timedelta(hours=random.randint(1, 24))).strftime('%Y-%m-%d %H:%M:%S')}', "
        f"{round(random.uniform(5, 50), 2)});"
        for _ in range(num_records)
    ]


# 24. FlightEmployeeAssignment Table
def generate_flight_employee_assignment_sql():
    return [
        f"INSERT INTO FlightEmployeeAssignment (FlightID, EmployeeID, AssignmentDate) VALUES "
        f"({random.randint(1, 199)}, {employee_id}, '{fake.date_this_year()}');"
        for employee_id in employee_data["AirplaneStaff"]
    ]

# 25. AirportAirlineOperations Table
def generate_airport_airline_operations_sql():
    return [
        f"INSERT INTO AirportAirlineOperations (AirportID, AirlineID, ContractStartDate, ContractEndDate, GateCount, TerminalsAssigned) VALUES "
        f"({random.randint(1, 25)}, {random.randint(100, 299)}, "
        f"'{(contract_start := fake.date_this_decade()).strftime('%Y-%m-%d')}', "
        f"'{(contract_end := contract_start + timedelta(days=random.randint(30, 3650))).strftime('%Y-%m-%d')}', "
        f"{random.randint(1, 10)}, '{random.choice(['Terminal 1', 'Terminal 2', 'Terminal 3'])}');"
        for _ in range(num_records)
    ]


# Generate SQL for all tables
table_generators = {
    "Airport": generate_airport_sql,
    "Airline": generate_airline_sql,
    "Passenger": generate_passenger_sql,
    "Phone": generate_phone_sql,
    "LoyaltyProgram": generate_loyalty_program_sql,
    "Employee": generate_employee_sql,
    "AirplaneStaff": generate_airplane_staff_sql,
    "Specialization": generate_specialization_sql,
    "Certification": generate_certification_sql,
    "SecurityStaff": generate_security_staff_sql,
    "GroundStaff": generate_ground_staff_sql,
    "GroundSupport": generate_ground_support_sql,
    "EquipmentAssignment": generate_equipment_assignment_sql,
    "RunwayManagement": generate_runway_management_sql,
    "Aircraft": generate_aircraft_sql,
    "Maintenance": generate_maintenance_sql,
    "Gate": generate_gate_sql,
    "Flight": generate_flight_sql,
    "FlightBooking": generate_flight_booking_sql,
    "BoardingPass": generate_boarding_pass_sql,
    "Luggage": generate_luggage_sql,
    "ParkingSlots": generate_parking_slots_sql,
    "Parking": generate_parking_sql,
    "FlightEmployeeAssignment": generate_flight_employee_assignment_sql,
    "AirportAirlineOperations": generate_airport_airline_operations_sql,
}

# Write SQL to file
def generate_sql_file(output_file="insert_statements.sql", num_records=200):
    with open(output_file, "w") as file:
        for table_name, generator in table_generators.items():
            print(f"Generating SQL for {table_name}...")
            # Check if the generator is generate_flight_sql and pass num_records
            if table_name == "Flight":  # Specific check for Flight table
                statements = generator(num_records)
            else:
                statements = generator()
            file.write("\n".join(statements) + "\n\n")
    print(f"SQL statements saved to {output_file}")


# Run the function to generate SQL
generate_sql_file()


In [ ]:
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()


def generate_flight_sql(num_records):
    flight_numbers = set()  # To ensure unique flight numbers
    inserts = []

    for _ in range(num_records):
        # Generate Source and Destination Airports, ensuring they're different
        source = random.randint(1, 25)
        dest = random.choice([id for id in range(1, 26) if id != source])

        # Generate other foreign keys
        airline = random.randint(100, 299)
        runway = random.randint(1, 50)
        gate = random.randint(1, 50)  # Assuming valid GateIDs exist from 1 to 50
        aircraft = random.randint(1, 199)

        # Generate a unique flight number
        flight_number = f"FL{random.randint(1000, 9999)}"
        while flight_number in flight_numbers:
            flight_number = f"FL{random.randint(1000, 9999)}"
        flight_numbers.add(flight_number)

        # Generate realistic Departure and Arrival times
        departure_time = fake.date_time_this_year()
        arrival_time = departure_time + timedelta(hours=random.randint(1, 12))

        # Generate flight status and set dependent attributes
        flight_status = random.choice(['Scheduled', 'Delayed', 'Cancelled', 'Completed'])

        if flight_status == 'Completed':
            actual_departure = departure_time
            actual_arrival = arrival_time
            delay_reason = None

        elif flight_status == 'Cancelled':
            actual_departure = None
            actual_arrival = None
            delay_reason = random.choice(['Weather Delay', 'Technical Issue', 'Air Traffic'])

        elif flight_status == 'Delayed':
            actual_departure = departure_time
            actual_arrival = arrival_time + timedelta(minutes=random.randint(30, 180))  # Random delay
            delay_reason = random.choice(['Weather Delay', 'Technical Issue', 'Air Traffic'])

        elif flight_status == 'Scheduled':
            actual_departure = None
            actual_arrival = None
            delay_reason = None

        # Create the SQL INSERT statement
        sql_statement = (
            f"INSERT INTO Flight (SourceAirportID, DestinationAirportID, AirlineID, RunwayID, AircraftID, GateID, "
            f"FlightNumber, FlightName, Capacity, DepartureTime, ArrivalTime, FlightPrice, FlightStatus, "
            f"ActualDepartureTime, ActualArrivalTime, DelayReason) VALUES "
            f"({source}, {dest}, {airline}, {runway}, {aircraft}, {gate}, '{flight_number}', '{fake.company()}', "
            f"{random.randint(100, 300)}, '{departure_time.strftime('%Y-%m-%d %H:%M:%S')}', "
            f"'{arrival_time.strftime('%Y-%m-%d %H:%M:%S')}', {round(random.uniform(100, 1500), 2)}, "
            f"'{flight_status}', "
            f"{'NULL' if not actual_departure else f"'{actual_departure.strftime('%Y-%m-%d %H:%M:%S')}'"}, "
            f"{'NULL' if not actual_arrival else f"'{actual_arrival.strftime('%Y-%m-%d %H:%M:%S')}'"}, "
            f"{'NULL' if not delay_reason else f"'{delay_reason}'"});"
        )

        # Print the generated SQL statement
        print(sql_statement)

        # Add the statement to the list of inserts
        inserts.append(sql_statement)

    return inserts

with open("flights.sql", "w") as file:
    file.write("\n".join(generate_flight_sql(200)))



In [ ]:
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()
num_records = 200
def generate_maintenance_sql():
    maintenance_sql = []
    for _ in range(num_records):
        maintenance_date = fake.date_this_year()  # Generate a random maintenance date in the current year
        maintenance_duration = random.randint(1, 8)  # Random duration in hours
        next_scheduled_date = fake.date_between(start_date=maintenance_date, end_date='+2y')  # Ensure it's after maintenance date

        maintenance_sql.append(
            f"INSERT INTO Maintenance (AircraftID, EquipmentID, MaintenanceDate, MaintenanceType, MaintenanceDuration, NextScheduledMaintenance, MaintenanceStatus) VALUES "
            f"({random.randint(1, 199)}, {random.randint(440, 639)}, '{maintenance_date}', '{random.choice(['Routine Check', 'Engine Check', 'Hydraulic System Maintenance'])}', "
            f"{maintenance_duration}, '{next_scheduled_date}', '{random.choice(['Pending', 'In Progress', 'Completed'])}');"
        )
    return maintenance_sql

# Generate the SQL statements
sql_statements = generate_maintenance_sql()

# Print the generated SQL for review
for sql in sql_statements:
    print(sql)


In [ ]:
import random

def generate_ground_support_sql(num_records=200):
    return [
        f"INSERT INTO GroundSupport (EquipmentType, GSM_Status, UsageDuration, MaintenanceFrequency) VALUES "
        f"('{random.choice(['Baggage Cart', 'Fuel Truck', 'Tow Tractor', 'Ground Power Unit'])}', "
        f"'{random.choice(['Operational', 'Out of Service'])}', "
        f"{random.randint(30, 300)}, "  # Usage duration in minutes (e.g., 30 to 300 minutes)
        f"'{random.choice(['Monthly', 'Yearly'])}');"
        for _ in range(num_records)
    ]

# Example Usage
sql_statements = generate_ground_support_sql(200)
for statement in sql_statements:
    print(statement)


In [ ]:
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()


def generate_flight_sql(num_records):
    flight_numbers = set()  # To ensure unique flight numbers
    inserts = []

    for _ in range(num_records):
        # Generate Source and Destination Airports, ensuring they're different
        source = random.randint(1, 25)
        dest = random.choice([id for id in range(1, 26) if id != source])

        # Generate other foreign keys
        airline = random.randint(100, 299)
        runway = random.randint(1, 50)
        gate = random.randint(1, 50)  # Assuming valid GateIDs exist from 1 to 50
        aircraft = random.randint(1, 199)

        # Generate a unique flight number
        flight_number = f"FL{random.randint(1000, 9999)}"
        while flight_number in flight_numbers:
            flight_number = f"FL{random.randint(1000, 9999)}"
        flight_numbers.add(flight_number)

        # Generate realistic Departure and Arrival times within the past 5 years
        departure_time = fake.date_time_between(start_date='-5y', end_date='now')
        arrival_time = departure_time + timedelta(hours=random.randint(1, 12))

        # Generate flight status and set dependent attributes
        flight_status = random.choice(['Scheduled', 'Delayed', 'Cancelled', 'Completed'])

        if flight_status == 'Completed':
            actual_departure = departure_time
            actual_arrival = arrival_time
            delay_reason = None

        elif flight_status == 'Cancelled':
            actual_departure = None
            actual_arrival = None
            delay_reason = random.choice(['Weather Delay', 'Technical Issue', 'Air Traffic'])

        elif flight_status == 'Delayed':
            actual_departure = departure_time
            actual_arrival = arrival_time + timedelta(minutes=random.randint(30, 180))  # Random delay
            delay_reason = random.choice(['Weather Delay', 'Technical Issue', 'Air Traffic'])

        elif flight_status == 'Scheduled':
            actual_departure = None
            actual_arrival = None
            delay_reason = None

        # Create the SQL INSERT statement
        sql_statement = (
            f"INSERT INTO Flight (SourceAirportID, DestinationAirportID, AirlineID, RunwayID, AircraftID, GateID, "
            f"FlightNumber, FlightName, Capacity, DepartureTime, ArrivalTime, FlightPrice, FlightStatus, "
            f"ActualDepartureTime, ActualArrivalTime, DelayReason) VALUES "
            f"({source}, {dest}, {airline}, {runway}, {aircraft}, {gate}, '{flight_number}', '{fake.company()}', "
            f"{random.randint(100, 300)}, '{departure_time.strftime('%Y-%m-%d %H:%M:%S')}', "
            f"'{arrival_time.strftime('%Y-%m-%d %H:%M:%S')}', {round(random.uniform(100, 1500), 2)}, "
            f"'{flight_status}', "
            f"{'NULL' if not actual_departure else f"'{actual_departure.strftime('%Y-%m-%d %H:%M:%S')}'"}, "
            f"{'NULL' if not actual_arrival else f"'{actual_arrival.strftime('%Y-%m-%d %H:%M:%S')}'"}, "
            f"{'NULL' if not delay_reason else f"'{delay_reason}'"});"
        )

        # Print the generated SQL statement
        print(sql_statement)

        # Add the statement to the list of inserts
        inserts.append(sql_statement)

    return inserts


# Generate and save the SQL statements to a file
with open("flights_past_5_years.sql", "w") as file:
    file.write("\n".join(generate_flight_sql(200)))
